<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.05-Hierarchical-Indexing.ipynb"><img align="left" src="https: //colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# 階層インデックス

ここまでは、Pandas の ``Series`` オブジェクトと ``DataFrame`` オブジェクトにそれぞれ格納されている 1 次元と 2 次元のデータに主に焦点を当ててきました。
多くの場合、これを超えて、より高次元のデータ (つまり、1 つまたは 2 つ以上のキーによってインデックス付けされたデータ) を格納すると便利です。
Pandas は、3 次元および 4 次元のデータをネイティブに処理する ``Panel`` および ``Panel4D`` オブジェクトを提供しますが ([Aside: Panel Data](#Aside:-Panel-Data) を参照)、実際のはるかに一般的なパターンは、*階層インデックス* を利用することです (また、 *multi-indexing* として知られている) を使用して、単一のインデックス内に複数のインデックス *レベル* を組み込みます。
このようにして、高次元のデータは、使い慣れた 1 次元の ``Series`` と 2 次元の ``DataFrame`` オブジェクト内でコンパクトに表すことができます。

このセクションでは、 ``MultiIndex`` オブジェクトの直接作成、多重インデックス化されたデータ全体のインデックス化、スライス、および統計の計算に関する考慮事項、およびデータの単純なインデックス表現と階層的にインデックス化された表現の間で変換するための便利なルーチンについて説明します。

標準のインポートから始めます。

In [1]:
import pandas as pd
import numpy as np

## 多重索引シリーズ

1 次元の ``Series`` 内で 2 次元のデータを表現する方法を考えることから始めましょう。
具体的には、各ポイントに文字と数字キーがある一連のデータを考えます。

### 悪い方法

2 つの異なる年の州に関するデータを追跡するとします。
既に説明した Pandas ツールを使用すると、単純に Python のタプルをキーとして使用したくなるかもしれません。

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

この索引付けスキームを使用すると、この複数の索引に基づいてシリーズを簡単に索引付けまたはスライスできます。

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

しかし、便利さはそこで終わります。たとえば、2010 年のすべての値を選択する必要がある場合は、それを実現するために面倒な (そして潜在的に遅い) 変更を行う必要があります。

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

これにより、望ましい結果が得られますが、Pandas で愛されるようになったスライス構文ほどクリーンではありません (大規模なデータセットに対して効率的ではありません)。

### より良い方法: Pandas MultiIndex
幸いなことに、Pandas はより良い方法を提供します。
タプルベースのインデックス作成は基本的に初歩的なマルチインデックスであり、Pandas ``MultiIndex`` 型は必要な操作の型を提供します。
次のように、タプルからマルチインデックスを作成できます。

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

``MultiIndex`` には、インデックス付けの複数の *レベル* が含まれていることに注意してください。この場合、州名と年、およびこれらのレベルをエンコードする各データ ポイントの複数の *ラベル* です。

この ``MultiIndex`` でシリーズのインデックスを再作成すると、データの階層表現が表示されます。

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

ここで ``Series`` 表現の最初の 2 列は複数のインデックス値を示し、3 番目の列はデータを示しています。
最初の列で一部のエントリが欠落していることに注意してください。このマルチインデックス表現では、空白のエントリはその上の行と同じ値を示します。

2 番目のインデックスが 2010 であるすべてのデータにアクセスするには、Pandas スライス表記を使用するだけです。

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

結果は、関心のあるキーだけを持つ単一インデックスの配列です。
この構文は、私たちが始めた自家製のタプルベースのマルチインデックス ソリューションよりもはるかに便利です (そして、操作ははるかに効率的です!)。
ここで、階層的に索引付けされたデータに対するこの種の索引付け操作についてさらに説明します。

### 追加次元としての MultiIndex

ここで別のことに気付くかもしれません: インデックスと列ラベルを持つ単純な ``DataFrame`` を使用して、同じデータを簡単に格納できたはずです。
実際、Pandas はこの同等性を念頭に置いて構築されています。 ``unstack()`` メソッドは、多重インデックス化された ``Series`` を従来のインデックス化された ``DataFrame`` にすばやく変換します:

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


当然、 ``stack()`` メソッドは逆の操作を提供します:

In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

これを見ると、なぜわざわざ階層的な索引付けを行う必要があるのか​​と疑問に思うかもしれません。
理由は簡単です: マルチインデックスを使用して 1 次元の ``Series`` 内で 2 次元のデータを表すことができたのと同じように、それを使用して ``Series' で 3 次元以上のデータを表すこともできます。 `` または ``DataFrame``.
マルチインデックスの各余分なレベルは、データの余分な次元を表します。このプロパティを利用することで、表現できるデータの種類をより柔軟に選択できます。具体的には、毎年、州ごとに別の人口統計データの列を追加したい場合があります (たとえば、18 歳未満の人口)。 ``MultiIndex`` を使用すると、これは ``DataFrame`` に別の列を追加するのと同じくらい簡単です:

In [10]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

さらに、[Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) で説明されているすべての ufunc およびその他の機能は、階層インデックスでも機能します。
上記のデータを基に、18 歳未満の人の割合を年ごとに計算します。

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


これにより、高次元のデータであっても簡単かつ迅速に操作および調査できます。

## マルチインデックスの作成方法

多重インデックスの ``Series`` または ``DataFrame`` を作成する最も簡単な方法は、単純に 2 つ以上のインデックス配列のリストをコンストラクタに渡すことです。例えば：

In [12]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.554233  0.356072
  2  0.925244  0.219474
b 1  0.441759  0.610054
  2  0.171495  0.886688

``MultiIndex`` を作成する作業はバックグラウンドで行われます。

同様に、適切なタプルをキーとして辞書を渡すと、Pandas はこれを自動的に認識し、デフォルトで ``MultiIndex`` を使用します。

In [13]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

それでも、明示的に ``MultiIndex`` を作成すると便利な場合があります。ここでは、これらのメソッドのいくつかを紹介します。

### 明示的な MultiIndex コンストラクター

インデックスの構築方法をより柔軟にするために、代わりに ``pd.MultiIndex`` で利用可能なクラス メソッド コンストラクタを使用できます。
たとえば、前に行ったように、各レベル内のインデックス値を与える配列の単純なリストから ``MultiIndex`` を構築できます。

In [14]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

各ポイントの複数のインデックス値を与えるタプルのリストから構築できます。

In [15]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

単一のインデックスのデカルト積から構築することもできます。

In [16]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

同様に、levels (各レベルで使用可能なインデックス値を含むリストのリスト) と labels (これらを参照するリストのリスト) を渡すことにより、内部エンコーディングを使用して MultiIndex を直接構築できます。ラベル):

In [17]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

これらのオブジェクトはいずれも、 Series または Dataframe を作成するときに index 引数として渡すか、既存の Series または Dataframe の reindex メソッドに渡すことができます。 ``データフレーム``.

### MultiIndex レベル名

``MultiIndex`` のレベルに名前を付けると便利な場合があります。
これは、上記の MultiIndex コンストラクタのいずれかに ``names`` 引数を渡すか、事後にインデックスの ``names`` 属性を設定することで実現できます:

In [18]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

より複雑なデータセットでは、これはさまざまなインデックス値の意味を追跡するのに役立つ方法です。

### 列のマルチインデックス

``DataFrame`` では、行と列は完全に対称であり、行が複数レベルのインデックスを持つことができるように、列も複数レベルを持つことができます。
以下を検討してください。これは、いくつかの (やや現実的な) 医療データのモックアップです。

In [19]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  38.7  32.0  36.7  35.0  37.2
     2      44.0  37.7  50.0  35.0  29.0  36.7
2014 1      30.0  37.4  39.0  37.8  61.0  36.9
     2      47.0  37.8  48.0  37.3  51.0  36.5

ここで、行と列の両方のマルチインデックスがどこで「非常に」便利になるかがわかります。
これは基本的に 4 次元データであり、次元は被験者、測定タイプ、年、および訪問数です。
これにより、たとえば、最上位の列に人物の名前でインデックスを付け、その人物の情報だけを含む完全な ``DataFrame`` を取得できます。

In [20]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      32.0  36.7
     2      50.0  35.0
2014 1      39.0  37.8
     2      48.0  37.3

多くの主題 (人、国、都市など) について、複数回にわたる複数のラベル付き測定値を含む複雑なレコードの場合、階層的な行と列を使用すると非常に便利です!

## MultiIndex のインデックス作成とスライス

``MultiIndex`` でのインデックス作成とスライスは直感的に行えるように設計されており、インデックスを追加された次元と考えると役立ちます。
最初に複数インデックスの ``Series`` のインデックスを見てから、複数インデックスの ``DataFrame`` を見ていきます。

### インデックス付きシリーズの乗算

先に見た州の人口の複数インデックスの ``Series`` を考えてみましょう:

In [21]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

複数の用語でインデックスを作成することにより、単一の要素にアクセスできます。

In [22]:
pop['California', 2000]

33871648

``MultiIndex`` は、*部分インデックス*、つまりインデックス内のレベルの 1 つだけをインデックスすることもサポートしています。
結果は、下位レベルのインデックスが維持された別の ``Series`` になります。

In [23]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

``MultiIndex`` がソートされている限り、部分的なスライスも利用できます ([Sorted and Unsorted Indices](#Sorted-and-unsorted-indices) の議論を参照してください):

In [24]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

ソートされたインデックスを使用すると、最初のインデックスに空のスライスを渡すことで、下位レベルで部分的なインデックス作成を実行できます。

In [25]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

他のタイプのインデックス作成と選択 ([Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) で説明) も機能します。たとえば、ブール マスクに基づく選択:

In [26]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

ファンシーインデックスに基づく選択も機能します。

In [27]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### インデックス付きの DataFrame を乗算する

多重インデックスの ``DataFrame`` も同様に動作します。
以前のおもちゃの医療 ``DataFrame`` を考えてみましょう:

In [28]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  38.7  32.0  36.7  35.0  37.2
     2      44.0  37.7  50.0  35.0  29.0  36.7
2014 1      30.0  37.4  39.0  37.8  61.0  36.9
     2      47.0  37.8  48.0  37.3  51.0  36.5

列は ``DataFrame`` のプライマリであり、多重インデックスの ``Series`` に使用される構文が列に適用されることに注意してください。
たとえば、簡単な操作で Guido の心拍数データを復元できます。

In [29]:
health_data['Guido', 'HR']

year  visit
2013  1        32.0
      2        50.0
2014  1        39.0
      2        48.0
Name: (Guido, HR), dtype: float64

また、単一インデックスの場合と同様に、[Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) で導入された loc 、 iloc 、および ix インデクサーを使用できます。例えば：

In [30]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      31.0  38.7
     2      44.0  37.7

これらのインデクサーは、基礎となる 2 次元データの配列のようなビューを提供しますが、 ``loc`` または ``iloc`` の個々のインデックスには、複数のインデックスのタプルを渡すことができます。例えば：

In [31]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        31.0
      2        44.0
2014  1        30.0
      2        47.0
Name: (Bob, HR), dtype: float64

これらのインデックス タプル内でスライスを操作するのは特に便利ではありません。タプル内でスライスを作成しようとすると、構文エラーが発生します。

In [32]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-32-8e3cc151e316>, line 1)

Python の組み込みの ``slice()`` 関数を使用して目的のスライスを明示的に構築することでこれを回避できますが、このコンテキストでのより良い方法は ``IndexSlice`` オブジェクトを使用することです。これは Pandas がまさにこの状況のた​​めに提供するものです。 .
例えば：

In [33]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,31.0,32.0,35.0
2014,1,30.0,39.0,61.0


多重インデックスの ``Series`` と ``DataFrame`` でデータを操作する方法はたくさんあります。この本の多くのツールと同様に、それらに慣れるための最善の方法はそれらを試してみることです!

## マルチインデックスの並べ替え

多重インデックス データを操作するための鍵の 1 つは、データを効果的に変換する方法を知ることです。
データセット内のすべての情報を保持しながら、さまざまな計算のために再配置する多くの操作があります。
``stack()`` メソッドと ``unstack()`` メソッドでこの簡単な例を見ましたが、階層インデックスと列の間のデータの再配置を細かく制御する方法は他にもたくさんあります。それらをここに。

### ソートされたインデックスとソートされていないインデックス

先ほど、注意事項について簡単に説明しましたが、ここではもっと強調する必要があります。
*インデックスがソートされていない場合、MultiIndex スライス操作の多くは失敗します。
ここでこれを見てみましょう。

インデックスが*辞書順でソートされていない*単純な乗算インデックス付きデータを作成することから始めます。

In [34]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.003001
      2      0.164974
c     1      0.741650
      2      0.569264
b     1      0.001693
      2      0.526226
dtype: float64

このインデックスの部分スライスを取得しようとすると、エラーが発生します。

In [35]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'KeyError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


エラー メッセージからは完全にはわかりませんが、これは MultiIndex が並べ替えられていないことが原因です。
さまざまな理由から、部分スライスやその他の同様の操作では、MultiIndex 内のレベルがソートされた (つまり、辞書順である) 必要があります。
Pandas には、この種の並べ替えを実行するための便利なルーチンが多数用意されています。例は ``DataFrame`` の ``sort_index()`` および ``sortlevel()`` メソッドです。
ここでは、最も単純な ``sort_index()`` を使用します:

In [36]:
data = data.sort_index()
data

char  int
a     1      0.003001
      2      0.164974
b     1      0.001693
      2      0.526226
c     1      0.741650
      2      0.569264
dtype: float64

この方法でソートされたインデックスを使用すると、部分スライスが期待どおりに機能します。

In [37]:
data['a':'b']

char  int
a     1      0.003001
      2      0.164974
b     1      0.001693
      2      0.526226
dtype: float64

### インデックスのスタックとアンスタック

前に簡単に説明したように、データセットを積み上げマルチインデックスから単純な 2 次元表現に変換することができ、オプションで使用するレベルを指定できます。

In [38]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [39]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


``unstack()`` の反対は ``stack()`` で、元の系列を復元するために使用できます:

In [40]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### インデックスの設定と再設定

階層データを再編成するもう 1 つの方法は、インデックス ラベルを列に変換することです。これは ``reset_index`` メソッドで実現できます。
人口辞書でこれを呼び出すと、以前はインデックスにあった情報を保持する *state* および *year* 列を持つ ``DataFrame`` が生成されます。
明確にするために、必要に応じて列表現のデータの名前を指定できます。

In [41]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


現実の世界でデータを扱う場合、生の入力データはこのようになることが多く、列の値から ``MultiIndex`` を構築すると便利です。
これは ``DataFrame`` の ``set_index`` メソッドで行うことができます。このメソッドは複数のインデックスを持つ ``DataFrame`` を返します:

In [42]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

実際には、このタイプの再索引付けは、現実世界のデータセットに遭遇した場合に役立つパターンの 1 つです。

## マルチインデックスでのデータ集計

Pandas には ``mean()``、``sum()``、``max()`` などの組み込みのデータ集計メソッドがあることを以前に見ました。
階層的に索引付けされたデータの場合、集計が計算されるデータのサブセットを制御する ``level`` パラメータを渡すことができます。

たとえば、健康データに戻りましょう。

In [43]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  38.7  32.0  36.7  35.0  37.2
     2      44.0  37.7  50.0  35.0  29.0  36.7
2014 1      30.0  37.4  39.0  37.8  61.0  36.9
     2      47.0  37.8  48.0  37.3  51.0  36.5

おそらく、毎年 2 回の訪問で測定値を平均したいと思います。これを行うには、調べたいインデックス レベル (この場合は年) に名前を付けます。

In [44]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     37.5  38.2  41.0  35.85  32.0  36.95
2014     38.5  37.6  43.5  37.55  56.0  36.70

さらに ``axis`` キーワードを利用することで、列のレベル間で平均を取ることもできます:

In [45]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,36.833333,37.000000
2014,46.000000,37.283333


このように、2 行で、毎年すべての訪問ですべての被験者が測定した平均心拍数と体温を見つけることができました。
この構文は、実際には [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb) で説明する ``GroupBy`` 機能へのショートカットです。
これはおもちゃの例ですが、実際のデータセットの多くは同様の階層構造を持っています。

## 余談: パネル データ

Pandas には、まだ説明していない基本的なデータ構造が他にもいくつかあります。つまり、 pd.Panel オブジェクトと pd.Panel4D オブジェクトです。
これらはそれぞれ、(1 次元) ``Series`` および (2 次元) ``DataFrame`` 構造の 3 次元および 4 次元の一般化と考えることができます。
``Series`` と ``DataFrame`` のデータのインデックス作成と操作に慣れれば、``Panel`` と ``Panel4D`` は比較的簡単に使用できます。
特に、[Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) で説明されている ix 、 loc 、および iloc インデクサーは、これらの高次元構造に容易に拡張できます。

これらのパネル構造については、このテキストではこれ以上説明しません。ほとんどの場合、複数のインデックス作成がより便利で、より高次元のデータの概念的に単純な表現であることがわかったからです。
さらに、パネル データは基本的に密なデータ表現ですが、マルチインデックスは基本的に疎なデータ表現です。
次元数が増えると、実世界のデータセットの大部分で高密度表現が非常に非効率になる可能性があります。
ただし、時折の特殊なアプリケーションでは、これらの構造が役立つ場合があります。
``Panel`` および ``Panel4D`` 構造について詳しく知りたい場合は、[Further Resources](03.13-Further-Resources.ipynb) にリストされているリファレンスを参照してください。

<!--ナビゲーション-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.05-Hierarchical-Indexing.ipynb"><img align="left" src="https: //colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
